# ETL_Part2

__!!!!!IMP!!!!: On the day when the timezone changes, we must implement a new logic. We utilize the current UTC time, but it's crucial not to mix data from before and after the time change. Otherwise, all data will advance by one hour. If you use the old UTC time, the data will be delayed by one hour.__

   _Portugal > Winter: "UTC +0" || Summers: "UTC +1" (+01:00 Paris now)_	

__Run the bellow 2 groups ( Libraries +USERNAME and DOWNLOAD FOLDER)__

 1 group will import the libraries, if some of it will not work, please open a new code cell and write: Example ( pip install shutill) and run it, this will install the library
 
 2 group will read you username and the folder where you have the downloads, you can have another then download, just ajust the variable " folder"

# Libraries

In [1]:
import os
import pandas as pd
import glob
import numpy as np
import time
import shutil
import re
import win32com.client
from pathlib import Path
from zipfile import ZipFile
from datetime import datetime, timedelta
import calendar
import win32com.client as win32
import subprocess
import warnings
import win32com.client as w3c
from dateutil.relativedelta import relativedelta
from openpyxl import load_workbook

warnings.filterwarnings('ignore')

# USERNAME and DOWNLOAD FOLDER

__!!!CHANGE THE  USER NAME, AND DOWNLOAD PATH IF NEEDED:__


In [2]:
#THE CODE SHOULD SHOW YOUR NAME:
username = os.getlogin()
print('Your username:'+ username)
#IF YOU DONT USE YOUR DOWNLOAD FOLDER CHANGE THE BELOW STRING TO YOUR FOLDER:
folder="Downloads"
downloads_folder_path = fr"C:\\Users\\{username}\\{folder}"
print('Folder where is your downloads:' + downloads_folder_path)
#Clean python cache
gen_py_path = fr"C:\Users\\{username}\\AppData\Local\Temp\gen_py"
shutil.rmtree(gen_py_path, ignore_errors=True)

Your username:bechara.9
Folder where is your downloads:C:\\Users\\bechara.9\\Downloads


--------------------------------------------------------------------------------------------------------------------------

# ( !!!MONDAY // WEDNESDAY!!!) Random Sampling ( TTR1, TTR2, LIVE)

__Link:__ 

https://datapower-va.bytelemon.com/bi/visit/7361807889996120069


 __Moderation Date:__ Extract last 4 weeks of data ( Monday to Sunday)  // __Press "Querry"__

 __Tab:__ Ops Record

 __Table:__ OPS Record SQA

__Number of rows:__ 1000.000

__Format:__ UTF-8 encoded CSV

In [3]:
#Destination Emea:
destination = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\New_Random Sampling_Detailed"

# List files in the downloads folder
files = os.listdir(downloads_folder_path)
week_file_count = {}

for file in files:
    if file.startswith("_OPS Record SQA -"):  
        source_path = os.path.join(downloads_folder_path, file)
        df = pd.read_csv(source_path, delimiter=",")  
        
        # Extract unique date
        unique_date = pd.to_datetime(df["Date"]).dt.date.unique()
        
        # Extract the date from the first row
        date_str = df["Date"].iloc[0]
        date = datetime.strptime(date_str, "%Y-%m-%d")
        week = date.strftime("2024 W%U")
        
        next_week_date = date + timedelta(days=8)
        week = next_week_date.strftime("2024 W%U")
        
        if week in week_file_count:
            week_file_count[week].append((file, unique_date))
        else:
            week_file_count[week] = [(file, unique_date)]



for week, files_list in week_file_count.items():
    for index, (file, unique_date) in enumerate(files_list, 1):
        source_path = os.path.join(downloads_folder_path, file)
        if len(files_list) == 1:
            new_name = f"Ops_Record - {week}.csv"
        else:
            new_name = f"Ops_Record - {week}_part{index}.csv"

        destination_path = os.path.join(destination, new_name)
        shutil.copy(source_path, destination_path)
        print(f"File copied to: {new_name}")

#This line will remove the files from your download files:
[os.remove(os.path.join(downloads_folder_path, filename)) for filename in os.listdir(downloads_folder_path)
 if filename.startswith("_OPS Record SQA -")]

File copied to: Ops_Record - 2024 W34.csv
File copied to: Ops_Record - 2024 W33.csv
File copied to: Ops_Record - 2024 W32.csv
File copied to: Ops_Record - 2024 W31.csv


[None, None, None, None]

----------------------------------------END of this code Group--------------------------------------------------------------


--------------------------------------------------------------------------------------------------------------------------

https://byteworks-va.bytelemon.com/v2/workhour/correct

# # Logic of the GA and UNO:
 

__In each process, we have Glidepaths files from TP QAs and raw data from the client. Then we utilize a spider script to bring specifc columns from the Glidepath, base on the task_id + Queue ID into the client's dataset.__

#    ( !!!MONDAY // WEDNESDAY!!!) GA GLIEPATHS Bot is doing this



----------TTR1 Glidepaths---------


__PT TTR1 Glidepath & Action Tracker:__
https://teleperformance.larksuite.com/sheets/shtusjLubYGiKkRYwhqySE39Hgg?sheet=XhmUPq

__FI TTR1 Glidepath & Action Tracker:__
https://teleperformance.larksuite.com/sheets/shtusIbCsBcNIGhr9EsqGg4FHte?sheet=PRHk1L

__UA TTR1 Glidepath & Action Tracker:__
https://teleperformance.larksuite.com/sheets/shtusJDpWNWcJc7esxbFcKHqowe?sheet=ZzgRZL

__HB TTR1 Glidepath & Action Tracker:__
https://teleperformance.larksuite.com/sheets/shtusiIjVTZVpOHItQzXmSvYr0g?sheet=ILdzs7


----------TTR2 Glidepaths---------

__2024-PT TTR2 Glidepath & Action Tracker:__
https://teleperformance.larksuite.com/sheets/MciUsGYmHh3iOFtJHSxum0uFswb?sheet=ECWE7s

__UA TTR2 Glidepath & Action tracker:__
https://teleperformance.larksuite.com/sheets/Ufkqsy2WuhHIrHtdzfmuGlEWsEf?sheet=thvKHO

__Step 1:__ Run the code below. It will simultaneously open all the glidepaths, you just need to wait for the pages to load. In case it doesn't work, you use the links above.

__GA:IMP REMOVE FILTERS before download__



__Step 2:__ The below code will read the files that you downloaded and move it to destination folder.

__IMP:__ Verify the output, it will display the names of the files that have been moved. The code will execute the move operation if at least one file is present.

In [3]:
#Ps: If the file is renamed with a different name, you can simply update the code to read the new filename from your folder and add a new destination line with the name that the file should have.
#This ensures that we won't need to modify the GA script.
#THIS CODE WILL READ THE DOWNLOADED FILES:
GP_PT_TTR1=fr"C:\Users\\{username}\\{folder}\PT TTR1 Glidepath & Action Tracker - GA Review (100).csv"
GP_FI_TTR1=fr"C:\Users\\{username}\\{folder}\FI TTR1 Glidepath & Action Tracker - GA Review (100).csv"
GP_UA_TTR1=fr"C:\Users\\{username}\\{folder}\UA TTR1 Glidepath & Action Tracker - GA Review (100).csv"
GP_HB_TTR1=fr"C:\Users\\{username}\\{folder}\HB TTR1 Glidepath & Action Tracker - GA Review (100).csv"
GP_PT_TTR2=fr"C:\Users\\{username}\\{folder}\TTR2 - GA - PT Masterfile 2024 - GA Review (100).csv"
GP_UA_TTR2=fr"C:\Users\\{username}\\{folder}\UA TTR2 Glidepath & Action Tracker - GA Review (100).csv"

#DESTINATION FOLDER:
GP_TTR1_EMEA=r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\TTR1 GA Glidepath"
GP_TTR2_EMEA=r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\TTR2 GA Glidepath"


#THIS WILL MOVE ALL THE ABOVE FILES, IF THE FILE IS NOT DOWNLOAED THE CODE WILL NOT RUN:
file_paths = [
    
     #(File dowloaded,destination)
    (GP_PT_TTR1, GP_TTR1_EMEA),
    (GP_FI_TTR1, GP_TTR1_EMEA),
    (GP_UA_TTR1, GP_TTR1_EMEA),
    (GP_HB_TTR1, GP_TTR1_EMEA),
    (GP_PT_TTR2, GP_TTR2_EMEA),
    (GP_UA_TTR2, GP_TTR2_EMEA)
]

try:
    for source, destination in file_paths:
        try:
            shutil.copy(source, destination)
            print(f"Moved file: {source} ")
        except FileNotFoundError:
            continue  # If file is missing, continue to the next one
    print("Files moved with success.")
except Exception as e:
    print("Fails", e)

Moved file: C:\Users\\bechara.9\\Downloads\PT TTR1 Glidepath & Action Tracker - GA Review (100).csv 
Moved file: C:\Users\\bechara.9\\Downloads\FI TTR1 Glidepath & Action Tracker - GA Review (100).csv 
Moved file: C:\Users\\bechara.9\\Downloads\UA TTR1 Glidepath & Action Tracker - GA Review (100).csv 
Moved file: C:\Users\\bechara.9\\Downloads\HB TTR1 Glidepath & Action Tracker - GA Review (100).csv 
Moved file: C:\Users\\bechara.9\\Downloads\TTR2 - GA - PT Masterfile 2024 - GA Review (100).csv 
Moved file: C:\Users\\bechara.9\\Downloads\UA TTR2 Glidepath & Action Tracker - GA Review (100).csv 
Files moved with success.


----------------------------------------END of this code Group--------------------------------------------------------------


#  (!!!MONDAY // WEDNESDAY!!!) GA DataPower // Client Data

### <del>__OLD DataPower GA Link__:https://datapower-va.bytelemon.com/bi/visit/7171442951608877061?immersive=1

__DataPower GA Link__:https://datapower-va.bytelemon.com/bi/visit/7325384900452941829?immersive=1

__TAB:__ GA Performance

__SHEET:__ GA 1.0

__FILTERS TOP:__  _Site:_ " TP_LIS " /// PRESS: "QUERRY" to applay the filters. ´

__Table:__ Last Table >> "Case Picker - Simulation"

__Filter on Table:__  Batch

__Extract:__ Last 2 weeks data available. 

__Format:__ CSV.  // 1000000 of rows
    
__IMP:__ Sometimes we have 2 files per week ( example- Week 06 2024 we had (2024-02-05 and 2024-02-06)
    Each  need just contain 1 date values, but the GA script will verify that for us 


In [4]:
# Destination directory (final folder)
destination = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\GA Sampling_new\Datapower"

# List files in the downloads folder
files = os.listdir(downloads_folder_path)
week_file_count = {}

# Função para transformar a string para o formato "2024-06-17"
def transform_date(batch):
    match = re.search(r'(\d{4})-W\d{2}\((\d{2})/(\d{2})~\d{2}/\d{2}\)', batch)
    if match:
        year = match.group(1)
        month = match.group(2)
        day = match.group(3)
        return f"{year}-{month}-{day}"
    else:
        return batch
    
for file in files:
    if file.startswith("_Case Picker"):
        source_path = os.path.join(downloads_folder_path, file)
        df = pd.read_csv(source_path, delimiter=",")
         # Verificar se a coluna existe antes de renomear
        df.rename(columns={'Batch (Date)': 'Batch'}, inplace=True)
        # Apply the transformation to the Batch column
        df["Batch"] = df["Batch"].apply(transform_date)
        df["Batch"] = df["Batch"].astype(str)
        # Extract the date from the first row
        date_str = df["Batch"].iloc[0]
        date = datetime.strptime(date_str, "%Y-%m-%d")
         # Calcular a semana ISO
        year, week, _ = date.isocalendar()
        week_str = f"{year} W{week:02d}"
        
        df["Batch"] = date.strftime("%Y-%m-%d")
        unique_date = date_str 

        if week in week_file_count:
            week_file_count[week_str].append((file, unique_date))
        else:
            week_file_count[week_str] = [(file, unique_date)]
        # Salvar o DataFrame com o mesmo nome do arquivo original
        output_path = os.path.join(downloads_folder_path, file)
        df.to_csv(output_path, index=False)
       


print("Week File Count Dictionary:")
print(week_file_count)
print("--------------------------------------------")

for week, files_list in week_file_count.items():
    for index, (file, unique_date) in enumerate(files_list, 1):
        source_path = os.path.join(downloads_folder_path, file)
        if len(files_list) == 1:
            new_name = f"_Case Picker - Simulation  - {week}.csv"
        else:
            new_name = f"_Case Picker - Simulation  - {week}_part{index}.csv"

        destination_path = os.path.join(destination, new_name)

        shutil.copy(source_path, destination_path)
        print(f"File copied to: {new_name}   // Unique date: {unique_date}")

#This line will remove the files from your download files:
[os.remove(os.path.join(downloads_folder_path, filename)) for filename in os.listdir(downloads_folder_path)
 if filename.startswith("_Case Picker")]


Week File Count Dictionary:
{'2024 W34': [('_Case Picker - Simulation - 2024-08-21 08-36-31.csv', '2024-08-19')], '2024 W33': [('_Case Picker - Simulation - 2024-08-21 08-36-52.csv', '2024-08-12')], '2024 W32': [('_Case Picker - Simulation - 2024-08-21 08-40-02.csv', '2024-08-05')]}
--------------------------------------------
File copied to: _Case Picker - Simulation  - 2024 W34.csv   // Unique date: 2024-08-19
File copied to: _Case Picker - Simulation  - 2024 W33.csv   // Unique date: 2024-08-12
File copied to: _Case Picker - Simulation  - 2024 W32.csv   // Unique date: 2024-08-05


[None, None, None]

# (SCRIPT):GA SCRIPT
__The next script will run th GA script based on the inputs we give:__

__!!!IMP: CHANGE THE INPUT FOR THE CORRECT WEEK NUMBER!!!!__



In [5]:
Script_path = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\10. Python scripts\NEW ETL\Aux Scripts\GA Sampling\GA Sampling_new.py"

#run the second input


Input1 = '2024 W34'
Result1 = subprocess.run(['python', Script_path], input=Input1, text=True)


Input2 = '2024 W33'
Result2 = subprocess.run(['python', Script_path], input=Input2, text=True)

Input3 = '2024 W32'
Result3 = subprocess.run(['python', Script_path], input=Input3, text=True)


print("Result 1:", Result1)
print("Result 2:", Result2)
print("Result 3:", Result3)

Result 1: CompletedProcess(args=['python', '\\\\emea.tpg.ads\\portugal\\Departments\\ITDEV\\PowerBI\\accounting\\business analysts\\01 - ba\\02 - projects\\tiktok\\10. Python scripts\\NEW ETL\\Aux Scripts\\GA Sampling\\GA Sampling_new.py'], returncode=0)
Result 2: CompletedProcess(args=['python', '\\\\emea.tpg.ads\\portugal\\Departments\\ITDEV\\PowerBI\\accounting\\business analysts\\01 - ba\\02 - projects\\tiktok\\10. Python scripts\\NEW ETL\\Aux Scripts\\GA Sampling\\GA Sampling_new.py'], returncode=0)
Result 3: CompletedProcess(args=['python', '\\\\emea.tpg.ads\\portugal\\Departments\\ITDEV\\PowerBI\\accounting\\business analysts\\01 - ba\\02 - projects\\tiktok\\10. Python scripts\\NEW ETL\\Aux Scripts\\GA Sampling\\GA Sampling_new.py'], returncode=0)


__RCA VALIDATION:__

The below code will verify the RCAs mapped on out DIM_RCA an them compare with the RCA mapped on the final GA file, 
If the data has wrong RCA the code will trigger an email to the QAs. 

In [6]:
#lista_emails=['pedro.esteves.rivera@teleperformance.com']
lista_emails=['divelly.rivera@teleperformance.com','dario.rocco@teleperformance.com','soraia.amarelinho@teleperformance.com',
      'alexia.dejesus@teleperformance.com','domenico.frisone@pt.teleperformance.com','vanessa.rubio@teleperformance.com','natalia.shkodenko@teleperformance.com',]
#----------STD_DIM-------------
#Location of STD_DIM:
DIM_RCA_path= r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\STD_DIMS_TTOK new.xlsx'
#Reading the STD_DIM and the sheet "DIM_RCA", and pushing just the column "RCA_ID":
DIM_RCA=pd.read_excel(DIM_RCA_path,sheet_name='DIM_RCA')[['RCA_ID']]
#Filtring the column for the data that start with " Golden Assessment":
DIM_RCA=DIM_RCA[DIM_RCA.RCA_ID.str.startswith('Golden Assessment/')]
#Trim the all values:
DIM_RCA['RCA_ID']= DIM_RCA['RCA_ID'].str.strip()
#Lower case all the values
DIM_RCA['RCA_ID']= DIM_RCA['RCA_ID'].str.lower()

#------End of manipulation of the dataframe STD_DIM----------

#Creation of a function that will read just the last 4 the final data of GA on folder GA_samplin_new, folder Final_version:
def get_df1(folder_path):
    df1 = pd.DataFrame()
    os.chdir(folder_path)
    filesGA = sorted((f for f in os.listdir() if f.endswith('.csv')), key=os.path.getmtime, reverse=True)[:4]
    for file in filesGA:
        print("Reading file:", file)  # Print the name of the file being read
        aux = pd.read_csv(file, error_bad_lines=False)
        df1 = df1.append(aux)
    return df1
#Read location of the Final version of GA:
folder_path =  r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\GA Sampling_new\Final_version"
#Creating a dataframe of GA data:
df1 = get_df1(folder_path)
#Column creation "RCA_ID" with a concat of a string with 3 other columns with "/" as separation:
df1['RCA_ID'] = 'Golden Assessment' + '/' + df1['Agree/Not agree'].astype(str) + '/' + df1['1st RCA'] + '/' + df1['2nd RCA']
#Change the "Batch" column to datetime and with a specific date format:
df1['Batch'] =  pd.to_datetime(df1['Batch'], format='%Y/%m/%d')
#Column creation with the week number, base on the batch number:
df1['Week'] = df1['Batch'].dt.isocalendar().week
#Column change type to string:
df1['RCA_ID'] = df1['RCA_ID'].astype(str)
#Trim the values of the columns:
df1['RCA_ID']= df1['RCA_ID'].str.strip()
#Lower case the values of the colum:
df1['RCA_ID']= df1['RCA_ID'].str.lower()
df1['1st RCA']= df1['1st RCA'].str.lower()
df1['2nd RCA']= df1['2nd RCA'].str.lower()
#Filter out the nan values from the column:
df1= df1.loc[df1['RCA_ID'] != 'nan']
#Creating a dataframe with a specific columns:
df1=df1[['Batch','Week','Simulation Queue Round','Market','Agree/Not agree','1st RCA','2nd RCA','RCA_ID']]
# Extract the column you want to compare
column_to_compare = 'RCA_ID'
# Check if values from df2's column_to_compare are in df1
df1['RCA Validation'] = df1[column_to_compare].isin(DIM_RCA[column_to_compare]).map({True: 'Values present', False: 'Wrong RCA'})
def validate_rca(row):
    if row['1st RCA'] != 'correct case' and pd.isnull(row['2nd RCA']):
        return 'Wrong RCA'
    else:
        return row['RCA Validation']
df1['RCA Validation'] = df1.apply(validate_rca, axis=1)
#New data set will be generated only with the values that are "Missing"
missing_data= df1.loc[df1['RCA Validation'] == 'Wrong RCA']
missing_data = missing_data.drop_duplicates(keep='first', inplace=False)
#Creating a dataframe with a specific columns:
missing_data=missing_data[['Batch','Week','Simulation Queue Round','Market','Agree/Not agree','1st RCA','2nd RCA','RCA Validation']]

print('----------------------------------------------')
#_____________________________________________________________

#If the variable "missing_data" will be empty the file will not be save in excel on downloads folder:
if not missing_data.empty:
    # Define the path for the output Excel file
    missing_data.to_excel(fr"C:\\Users\\{username}\\{folder}\\GA1.0_RCA_validation.xlsx",index=False)
    
#_________Trigger an e-mail if the a Missing_data will be generated__________   

    # Create an Outlook instance
    ol = win32com.client.Dispatch("outlook.application")
    olmailitem = 0x0  # Mail item
    newmail = ol.CreateItem(olmailitem)
    newmail.Subject = 'GA 1.0 RCA with mistakes'
    newmail.To = ';'.join(lista_emails)  # Replace with the recipient's email address
    newmail.CC = "Data-Analytics-TikTok-Lisbon@pt.teleperformance.com"  # Replace with the CC email address if needed
    newmail.Body = '''\
Hello Team,

Could you please correct the RCA of GA 1.0 that are incorrect on the attached file.

Please let us know when all the errors will be correct, so we can extract all the data again.


Thank you.

'''
    # Attach the Excel file to the email
    attach1 = fr"C:\\Users\\{username}\\{folder}\\GA1.0_RCA_validation.xlsx"
    attach1 = newmail.Attachments.Add(attach1)
    # To display the mail before sending it
    # newmail.Display()

    # Send the email
    newmail.Send()
    
    print("Incorrect RCA found, the email will be send.")
else:
    print("All the RCAs are correct")





Reading file: QA_Case Picker - Simulation  - 2024 W32.csv
Reading file: QA_Case Picker - Simulation  - 2024 W33.csv
Reading file: QA_Case Picker - Simulation  - 2024 W34.csv
Reading file: QA_Case Picker - Simulation  - 2024 W31.csv
----------------------------------------------
All the RCAs are correct


----------------------------------------END of this code Group--------------------------------------------------------------


# ---------------------------------------------------------------------------------------------

# (!!!MONDAY // WED!!!)   UNO Project GA- Glidepaths

__Download glidepath file for the LOB(ttr1 and ttr2):__


__TTR1 IT // TAB: GA Review(100) GA 2.0__  https://teleperformance.larksuite.com/sheets/GR5zsTOhYhlcCetrj2auysbRsff?sheet=tux0nD

__TTR2 IT // TAB: GA Review(100) GA 2.0__  https://teleperformance.larksuite.com/sheets/HUnHsthM3hFsPvt90U2u2S4zsvg?sheet=d55kza

__TTR1 FR Link:TAB:Sample Cases__    https://teleperformance.larksuite.com/sheets/shtusRvfqaTzYkEzWeBeeCXNl4c?sheet=sbfUJT

__TTR1 FR Creator Link:TAB:Sample Cases__ https://teleperformance.larksuite.com/sheets/shtus1Juf67YEF9ZXlx0jjVSJ2d?sheet=HnwcUG

__TTR1 NL Link:TAB:Sample Cases__ https://teleperformance.larksuite.com/sheets/shtus7tLBP6YaA73FmeLO7aGnTh?sheet=Kh9ZTH

__TTR1 ES Link:TAB:Sample Cases__ https://teleperformance.larksuite.com/sheets/shtusbL94YqOzSPResz49UNwHYM?sheet=scRTg8

__TTR2 FR Link:TAB:Sample Cases__ https://teleperformance.larksuite.com/sheets/WeuXsloKmh7g1ati1dbuyyVOs2g?sheet=bUgE81

__SPS IT__ https://teleperformance.larksuite.com/sheets/BMmGs7rfghlgehtMwHMuMTkMsCf?sheet=EfOtFR

__SPS FR__  https://teleperformance.larksuite.com/sheets/BMmGs7rfghlgehtMwHMuMTkMsCf?sheet=RAS00L

__SPS NL__ https://teleperformance.larksuite.com/sheets/BMmGs7rfghlgehtMwHMuMTkMsCf?sheet=f910b9
       



__Step 1 :__ Run the below code to move the Glidepath to EMEA folder.


In [4]:
#Glidepath location:
UNO_GA_IT_TTR2= fr"C:\Users\\{username}\\{folder}\TTR2 - UNO Project - Masterfile - GA Review (100) GA 2.0.csv"
UNO_GA_IT_TTR1= fr"C:\Users\\{username}\\{folder}\TTR1 - UNO Project - Masterfile - GA Review (100) GA 2.0.csv"
UNO_GA_TTR1FR= fr"C:\Users\\{username}\\{folder}\FR TTR1 Glidepath & Action Tracker - GA Review (100) GA 2.0.csv"
UNO_GA_TTR1NL = fr"C:\Users\\{username}\\{folder}\NL TTR1 Glidepath & Action Tracker - GA Review (100) GA 2.0.csv"
UNO_GA_CreatorFR= fr"C:\Users\\{username}\\{folder}\Glidepath Creator TTR1 FR - GA Review (100) GA 2.0.csv"
UNO_GA_TTR1ES= fr"C:\Users\\{username}\\{folder}\ES TTR1 Glidepath & Action Tracker - GA Review (100) GA 2.0.csv"
UNO_GA_TTR1= fr"C:\Users\\{username}\\{folder}\NL TTR1 Glidepath & Action Tracker - GA Review (100) GA 2.0.csv"
Uno_GA_TTR2 = fr"C:\Users\\{username}\\{folder}\TTR2 - GA - FR Masterfile  - GA Review (100) GA 2.0.csv"
Uno_GA_SPS_IT =fr"C:\Users\\{username}\\{folder}\SPS GA Masterfile - IT GA Review (100) GA 2.0.csv"
Uno_GA_SPS_FR = fr"C:\Users\\{username}\\{folder}\SPS GA Masterfile - FR GA Review (100) GA 2.0.csv"
Uno_GA_SPS_NL = fr"C:\Users\\{username}\\{folder}\SPS GA Masterfile - NL GA Review (100) GA 2.0.csv"
#File destination:
UNO_Glidepath_location=r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\UNO Project\Glidepath files"

#This code will mode the file to the destination folder:
try:
    # Move the files from EMEA folder to DROPS folder 

    shutil.copy(UNO_GA_IT_TTR2, UNO_Glidepath_location)
    shutil.copy(UNO_GA_IT_TTR1, UNO_Glidepath_location)
    shutil.copy(UNO_GA_TTR1NL,UNO_Glidepath_location)
    shutil.copy(UNO_GA_TTR1FR, UNO_Glidepath_location)
    shutil.copy(UNO_GA_CreatorFR, UNO_Glidepath_location)
    shutil.copy(UNO_GA_TTR1ES, UNO_Glidepath_location)
    shutil.copy(UNO_GA_TTR1, UNO_Glidepath_location)
    shutil.copy(Uno_GA_TTR2, UNO_Glidepath_location)
    shutil.copy(Uno_GA_SPS_IT, UNO_Glidepath_location)
    shutil.copy(Uno_GA_SPS_FR, UNO_Glidepath_location)
    shutil.copy(Uno_GA_SPS_NL, UNO_Glidepath_location)

    
    print("Files moveed with success.")
except Exception as e:
    print("Fails", e)

Files moveed with success.


----------------------------------------END of this code Group--------------------------------------------------------------


#  (!!!MONDAY // WED!!!)UNO Project(GA SAMPLING)- Client Data // RUN THE UNO SCRIPT: CODE AFTER TTR1 CLIENT DATA 



__LINK:__ https://datapower-va.bytelemon.com/bi/visit/7325384900452941829?immersive=1
__TAB:__ GA Performance

__SHEET:__ GA 2.0

__FILTERS TOP:__  

         __Site:__ " TP_LIS " /// PRESS: "QUERRY" to applay the filters.

         __Batch Date__ ( files from saturday to  Friday) This how the client separates the weeks data. 

__Table:__ penultimate Table >> "Case Picker - Simulation"

__Extract:__ Last 2 weeks data available. 

__Format:__ CSV.  // 1000000 of rows

Download client files, LAST 2 WEEKS DATA.

__Run the bellow code:__

--------------------------------------------------------------------------------------------------------------------------

#Client data

In [8]:
#THIS CODE WILL ADAPT THE FILE NAME AND SEND THE FILE TO THE FOLDER DESTINATION:
destination = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\UNO Project\UNO_GA"


# List files in the downloads folder
files = os.listdir(downloads_folder_path)
week_file_count = {}
def transform_date(batch):
    match = re.search(r'(\d{4})-W\d{2}\((\d{2})/(\d{2})~\d{2}/\d{2}\)', batch)
    if match:
        year = match.group(1)
        month = match.group(2)
        day = match.group(3)
        return f"{year}-{month}-{day}"
    else:
        return batch
for file in files:
    if file.startswith("_Case Picker"):  
        source_path = os.path.join(downloads_folder_path, file)
        df = pd.read_csv(source_path, delimiter=",")
        df.rename(columns={'Batch(Date)': 'Batch'}, inplace=True)
        # Apply the transformation to the Batch column
        df["Batch"] = df["Batch"].apply(transform_date)
        df["Batch"] = df["Batch"].astype(str)
        # Extract the date from the first row
        date_str = df["Batch"].iloc[0]
        date = datetime.strptime(date_str, "%Y-%m-%d")  
        
         # Calcular a semana ISO
        year, week, _ = date.isocalendar()
        week_str = f"{year} W{week:02d}"
        
        df["Batch"] = date.strftime("%Y-%m-%d")
        unique_date = date_str 
        
        if week in week_file_count:
            week_file_count[week_str].append((file, unique_date))
        else:
            week_file_count[week_str] = [(file, unique_date)]
        # Salvar o DataFrame com o mesmo nome do arquivo original
        output_path = os.path.join(downloads_folder_path, file)
        df.to_csv(output_path, index=False)

print("Week File Count Dictionary:")
print(week_file_count)
print("--------------------------------------------------------------------------")

for week, files_list in week_file_count.items():
    for index, (file, unique_date) in enumerate(files_list, 1):
        source_path = os.path.join(downloads_folder_path, file)
        if len(files_list) == 1:
            new_name = f"_Case Picker - Simulation  - {week}.csv"
        else:
            new_name = f"_Case Picker - Simulation  - {week}_part{index}.csv"

        destination_path = os.path.join(destination, new_name)
        shutil.copy(source_path, destination_path)
        print(f"File copied to: {new_name}   // Unique date: {unique_date}")


print("-----------------------------------------------------------------------------")

#This line will remove the files from your download files:
[os.remove(os.path.join(downloads_folder_path, filename)) for filename in os.listdir(downloads_folder_path)
 if filename.startswith("_Case Picker")]

Week File Count Dictionary:
{'2024 W31': [('_Case Picker - Simulation - 2024-08-21 08-56-25.csv', '2024-08-03')]}
--------------------------------------------------------------------------
File copied to: _Case Picker - Simulation  - 2024 W31.csv   // Unique date: 2024-08-03
-----------------------------------------------------------------------------


[None]


# (SCRIPT): UNO SCRIPT
__The next script will run th UNO script base on the inputs we give:__

__!!!IMP: CHANGE THE INPUT TO THE CORRECT WEEK NUMBER, THE SAME AS THE WEEKS IN THE ABOVE CODE. !!!!__



In [9]:
Script_path = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\10. Python scripts\NEW ETL\Aux Scripts\UNO Sampling\UNO Sampling_new.py"


#run the second input


Input4 = '2024 W31'
Result4 = subprocess.run(['python', Script_path], input=Input4, text=True)

Input5 = '2024 W32'
Result5 = subprocess.run(['python', Script_path], input=Input5, text=True)

Input8 = '2024 W33'
Result8 = subprocess.run(['python', Script_path], input=Input8, text=True)

print("Result 1:", Result1)
print("Result 2:", Result2)
print("Result 3:", Result3)


Result 1: CompletedProcess(args=['python', '\\\\emea.tpg.ads\\portugal\\Departments\\ITDEV\\PowerBI\\accounting\\business analysts\\01 - ba\\02 - projects\\tiktok\\10. Python scripts\\NEW ETL\\Aux Scripts\\GA Sampling\\GA Sampling_new.py'], returncode=0)
Result 2: CompletedProcess(args=['python', '\\\\emea.tpg.ads\\portugal\\Departments\\ITDEV\\PowerBI\\accounting\\business analysts\\01 - ba\\02 - projects\\tiktok\\10. Python scripts\\NEW ETL\\Aux Scripts\\GA Sampling\\GA Sampling_new.py'], returncode=0)
Result 3: CompletedProcess(args=['python', '\\\\emea.tpg.ads\\portugal\\Departments\\ITDEV\\PowerBI\\accounting\\business analysts\\01 - ba\\02 - projects\\tiktok\\10. Python scripts\\NEW ETL\\Aux Scripts\\GA Sampling\\GA Sampling_new.py'], returncode=0)


In [11]:
#lista_emails=['pedro.esteves@teleperformance.com']
lista_emails=['divelly.rivera@teleperformance.com','dario.rocco@teleperformance.com','soraia.amarelinho@teleperformance.com',
      'alexia.dejesus@teleperformance.com','domenico.frisone@pt.teleperformance.com','vanessa.rubio@teleperformance.com','natalia.shkodenko@teleperformance.com',]
#----------STD_DIM-------------
#Location of STD_DIM:
DIM_RCA_path= r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\STD_DIMS_TTOK new.xlsx'
#Reading the STD_DIM and the sheet "DIM_RCA", and pushing just the column "RCA_ID":
DIM_RCA=pd.read_excel(DIM_RCA_path,sheet_name='DIM_RCA')[['RCA_ID']]
#Filtring the column for the data that start with " Golden Assessment":
DIM_RCA=DIM_RCA[DIM_RCA.RCA_ID.str.startswith('Golden Assessment/')]
#Trim the all values:
DIM_RCA['RCA_ID']= DIM_RCA['RCA_ID'].str.strip()
#Lower case all the values
DIM_RCA['RCA_ID']= DIM_RCA['RCA_ID'].str.lower()

#------End of manipulation of the dataframe STD_DIM----------

#Creation of a function that will read just the last 4 the final data of GA on folder GA_samplin_new, folder Final_version:
def get_df1(folder_path):
    df1 = pd.DataFrame()
    os.chdir(folder_path)
    filesGA = sorted((f for f in os.listdir() if f.endswith('.csv')), key=os.path.getmtime, reverse=True)[:4]
    for file in filesGA:
        print("Reading file:", file)  # Print the name of the file being read
        aux = pd.read_csv(file, error_bad_lines=False)
        df1 = df1.append(aux)
    return df1
#Read location of the Final version of GA:
folder_path =  r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\UNO Project\Final_version"
#Creating a dataframe of GA data:
df1 = get_df1(folder_path)
#Column creation "RCA_ID" with a concat of a string with 3 other columns with "/" as separation:
df1['RCA_ID'] = 'Golden Assessment' + '/' + df1['Agree/Not agree'].astype(str) + '/' + df1['1st RCA'].astype(str) + '/' + df1['2nd RCA']

#Read the DIM_LOB
lob=pd.read_excel(r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\STD_DIMS_TTOK new.xlsx',sheet_name='DIM_LOB')
lob['QUEUE_ID_L7']=lob['QUEUE_ID_L7'].astype(str)
df1["Queue ID_GA"]=df1["Queue ID_GA"].astype(str)
dim_lob=lob.rename(columns ={"QUEUE_ID_L7":"Queue ID_GA"})
dim_lob["Queue ID_GA"]=dim_lob["Queue ID_GA"].str.strip()   

df1 = pd.merge(df1, dim_lob[['Queue ID_GA', 'QUEUE_NAME_L8']].drop_duplicates(subset=['Queue ID_GA']), on='Queue ID_GA', how='left')



df1['RCA_ID'] = df1['RCA_ID'].astype(str)
#Trim the values of the columns:
df1['RCA_ID']= df1['RCA_ID'].str.strip()
#Lower case the values of the colum:
df1['RCA_ID']= df1['RCA_ID'].str.lower()
df1['1st RCA']= df1['1st RCA'].str.lower()
df1['2nd RCA']= df1['2nd RCA'].str.lower()
#Filter out the nan values from the column:
df1= df1.loc[df1['RCA_ID'] != 'nan']

#df1.to_csv(r"C:\Users\esteves.155\Downloads\rca_test.csv")
#Creating a dataframe with a specific columns:
df1=df1[['Batch','Market Name_GA','QUEUE_NAME_L8','Agree/Not agree','1st RCA','2nd RCA','RCA_ID']]
# Extract the column you want to compare
column_to_compare = 'RCA_ID'
# Check if values from df2's column_to_compare are in df1
df1['RCA Validation'] = df1[column_to_compare].isin(DIM_RCA[column_to_compare]).map({True: 'Values present', False: 'Wrong RCA'})
def validate_rca(row):
    if row['1st RCA'] != 'correct case' and pd.isnull(row['2nd RCA']):
        return 'Wrong RCA'
    else:
        return row['RCA Validation']
df1['RCA Validation'] = df1.apply(validate_rca, axis=1)
#New data set will be generated only with the values that are "Missing"
missing_data= df1.loc[df1['RCA Validation'] == 'Wrong RCA']
missing_data = missing_data.drop_duplicates(keep='first', inplace=False)
#Creating a dataframe with a specific columns:
missing_data=missing_data[['Batch','Market Name_GA','QUEUE_NAME_L8','Agree/Not agree','1st RCA','2nd RCA','RCA Validation']]


print('----------------------------------------------')
#_____________________________________________________________

#If the variable "missing_data" will be empty the file will not be save in excel on downloads folder:
if not missing_data.empty:
    # Define the path for the output Excel file
    missing_data.to_excel(fr"C:\\Users\\{username}\\{folder}\\GA2.0_RCA_validation.xlsx",index=False)
    
#_________Trigger an e-mail if the a Missing_data will be generated__________   

    # Create an Outlook instance
    ol = win32com.client.Dispatch("outlook.application")
    olmailitem = 0x0  # Mail item
    newmail = ol.CreateItem(olmailitem)
    newmail.Subject = 'GA 2.0 UNO RCA with mistakes'
    newmail.To = ';'.join(lista_emails)  # Replace with the recipient's email address
    newmail.CC = 'Data-Analytics-TikTok-Lisbon@pt.teleperformance.com'  # Replace with the CC email address if needed
    newmail.Body = '''\
Hello Team,

Could you please correct the RCA of GA2.0 that are incorrect on the attached file.

Please let us know when all the errors will be correct, so we can extract all the data again.


Thank you.

'''
    # Attach the Excel file to the email
    attach1 = fr"C:\\Users\\{username}\\{folder}\\GA2.0_RCA_validation.xlsx"
    attach1 = newmail.Attachments.Add(attach1)
    # To display the mail before sending it
    # newmail.Display()

    # Send the email
    newmail.Send()
    
    print("Incorrect RCA found, the email will be send.")
else:
    print("All the RCAs are correct")



Reading file: UNO_Moderator Error Case Picker_2024 W33.csv
Reading file: UNO_Moderator Error Case Picker_2024 W32.csv
Reading file: UNO_Moderator Error Case Picker_2024 W31.csv
Reading file: UNO_Moderator Error Case Picker_2024 W30.csv
----------------------------------------------
Incorrect RCA found, the email will be send.


----------------------------------------END of this code Group--------------------------------------------------------------


#  (!!!MONDAY) CGVR Quality

__Link__:https://datapower-va.bytelemon.com/bi/visit/7325384900452941829?immersive=1

__TAB:__ TCOR/CVGR

__SHEET:__ CGVR

__FILTERS TOP:__  _Site:_ " TP_LIS " // Moderation Date: from sunday to saturday

__Table:__ CGVR Case Picker

__Extract:__ Last 2 weeks data available (last and previous). 

__Format:__ CSV.  // 1000000 of rows 

In [3]:
# Destinos para os arquivos
destination1 = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\CCHA\CGVR"
destination2 = fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\CCHA\CGVR"

def extract_week_from_mod_resolve_time(mod_resolve_time):
    match = re.search(r'(\d{4}-W\d{2})\(\d{2}/\d{2}~\d{2}/\d{2}\)', mod_resolve_time)
    if match:
        # Retirar o hífen e substituir por espaço
        return match.group(1).replace('-', ' ')
    else:
        return None

# Caminho para a pasta de downloads tradicional
downloads_folder_path = os.path.join(os.path.expanduser('~'), 'Downloads')
files = os.listdir(downloads_folder_path)

# Listar arquivos que começam com "_CGVR Case Picker"
for file in files:
    if file.startswith("_CGVR Case Picker"):  
        source_path = os.path.join(downloads_folder_path, file)
        df = pd.read_csv(source_path, delimiter=",")

        # Extrair a semana da primeira linha da coluna "Mod. Resolve Time"
        week_str = None
        for mod_resolve_time in df["Mod. Resolve Time"]:
            week_str = extract_week_from_mod_resolve_time(mod_resolve_time)
            if week_str:
                break

        if week_str:
            # Formatar o nome do novo arquivo corretamente
            new_name = f"_CGVR Case Picker  - {week_str}.csv"
            destination_path1 = os.path.join(destination1, new_name)
            destination_path2 = os.path.join(destination2, new_name)
            
            # Substituir arquivo existente, se já estiver presente (Destino 1)
            if os.path.exists(destination_path1):
                os.remove(destination_path1)
            shutil.copy2(source_path, destination_path1)
            print(f"File copied to: {destination_path1}")
            
            # Substituir arquivo existente, se já estiver presente (Destino 2)
            if os.path.exists(destination_path2):
                os.remove(destination_path2)
            shutil.copy2(source_path, destination_path2)
            print(f"File copied to: {destination_path2}")
            
        else:
            print(f"Could not extract week from: {file}")

print("Process completed.")


File copied to: \\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\CCHA\CGVR\_CGVR Case Picker  - 2024 W33.csv
File copied to: C:\Users\\bechara.9\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\CCHA\CGVR\_CGVR Case Picker  - 2024 W33.csv
Process completed.


#  (!!!MONDAY) TCOR Quality

__Link__:https://datapower-va.bytelemon.com/bi/visit/7325384900452941829?immersive=1

__TAB:__ TCOR/CVGR

__SHEET:__ TCOR

__FILTERS TOP:__  _Site:_ " TP_LIS " // Moderation Date: from sunday to saturday

__Table:__ TCOR Quality Case Picker

__Extract:__ Last 2 weeks data available (last and previous). 

__Format:__ CSV.  // 1000000 of rows 

__Filename:__  manually rename file to = _TCOR Quality Case Picker  - AAAA W


In [6]:
destination1 = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\CCHA\TCOR"
destination2 = fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\CCHA\TCOR"

# Caminho para a pasta de downloads tradicional
downloads_folder_path = os.path.join(os.path.expanduser('~'), 'Downloads')
files = os.listdir(downloads_folder_path)

# List files that start with "_TCOR Quality Case Picker"
for file in files:
    if file.startswith("_TCOR Quality Case Picke"):  
        source_path = os.path.join(downloads_folder_path, file)
        
        # Caminho para o primeiro destino
        destination_path1 = os.path.join(destination1, file)
        
        # Substituir arquivo existente, se já estiver presente (Primeiro destino)
        if os.path.exists(destination_path1):
            os.remove(destination_path1)
        
        # Copiar o arquivo para o primeiro destino
        shutil.copy2(source_path, destination_path1)
        print(f"File copied to: {destination_path1}")
        
        # Caminho para o segundo destino
        destination_path2 = os.path.join(destination2, file)
        
        # Substituir arquivo existente, se já estiver presente (Segundo destino)
        if os.path.exists(destination_path2):
            os.remove(destination_path2)
        
        # Copiar o arquivo para o segundo destino
        shutil.copy2(source_path, destination_path2)
        print(f"File copied to: {destination_path2}")

print("Process completed.")


File copied to: \\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\CCHA\TCOR\_TCOR Quality Case Picker  - 2024 W33.csv
File copied to: C:\Users\\bechara.9\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\CCHA\TCOR\_TCOR Quality Case Picker  - 2024 W33.csv
Process completed.
